DSE CODING PROJECT


Importing Libraries

In [1]:
import pandas as pd
import geopandas as gpd 
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def load_data():
    data = pd.read_csv(r"C:\Users\greta\OneDrive\Desktop\GlobalLandTemperaturesByMajorCity.csv")
    data['Year'] = pd.to_datetime(data['dt']).dt.year  # Extract year for filtering
    data['Temperature'] = data['AverageTemperature']
    return data

In [4]:
# Function to filter data by selected year
def filter_data_by_year(data, year):
    return data[data['Year'] == year]